In [3]:
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds


2023-08-17 11:35:54.245285: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-17 11:35:54.558024: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-17 11:35:55.325933: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:
2023-08-17 11:35:55.326107: W tensorflow/compiler/xla/stream_exe

In [2]:
pt2en_train = tfds.load('ted_hrlr_translate/pt_to_en', split='train', as_supervised=True)
for pt, en in pt2en_train.take(1):
  print(pt.numpy().decode('utf-8'))
  print(en.numpy().decode('utf-8'))

2023-08-17 10:59:05.248578: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/ediddev/.local/lib/python3.8/site-packages/cv2/../../lib64:/usr/local/cuda/lib64:
2023-08-17 10:59:05.248651: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-08-17 10:59:05.248680: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (BOBO-CODE): /proc/driver/nvidia/version does not exist
2023-08-17 10:59:05.249238: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebu

e quando melhoramos a procura , tiramos a única vantagem da impressão , que é a serendipidade .
and when you improve searchability , you actually take away the one advantage of print , which is serendipity .


2023-08-17 10:59:05.576452: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [30]:
"""Task 0. Dataset"""


class Dataset:
    """loads and preps a dataset for machine translation"""
    def __init__(self, batch_size, max_len) -> None:
        self.data_train = tfds.load('ted_hrlr_translate/pt_to_en',
                                    split='train',
                                    as_supervised=True)
        self.data_valid = tfds.load('ted_hrlr_translate/pt_to_en',
                                    split='validation',
                                    as_supervised=True)
        self.max_len = max_len
        self.batch_size = batch_size
        self.tokenizer_pt, self.tokenizer_en = self.tokenize_dataset(self.data_train)

        self.data_train = self.data_train.map(self.tf_encode)
        self.data_valid = self.data_valid.map(self.tf_encode)

        def filter_max_length(x, y, max_length=self.max_len):
            """filter method"""
            return tf.logical_and(tf.size(x) <= max_length,
                                  tf.size(y) <= max_length)
        self.data_train = self.data_train.filter(filter_max_length)
        self.data_train = self.data_train.cache()
        self.data_train = self.data_train.shuffle(2**15, reshuffle_each_iteration=True).padded_batch(self.batch_size)
        self.data_train = self.data_train.prefetch(tf.data.experimental.AUTOTUNE)

        self.data_valid = self.data_valid.filter(filter_max_length).padded_batch(self.batch_size)

    def tokenize_dataset(self, data):
        """creates sub-word tokenizers for our dataset"""
        Subword = tfds.deprecated.text.SubwordTextEncoder
        tokenizer_pt = Subword.build_from_corpus((pt.numpy() for pt, _ in data),
                                                 target_vocab_size=2**15)
        tokenizer_en = Subword.build_from_corpus((en.numpy() for _, en in data),
                                                 target_vocab_size=2**15)
        return tokenizer_pt, tokenizer_en

    def encode(self, pt, en):
        """encodes a translation into tokens"""
        pt_vsize = self.tokenizer_pt.vocab_size
        en_vsize = self.tokenizer_en.vocab_size
        pt_tokens = [pt_vsize] + self.tokenizer_pt.encode(pt.numpy()) + \
                    [pt_vsize + 1]
        en_tokens = [en_vsize] + self.tokenizer_en.encode(en.numpy()) + \
                    [en_vsize + 1]
        return pt_tokens, en_tokens

    def tf_encode(self, pt, en):
        """acts as a tensorflow wrapper for the encode instance method"""
        pt_lang, en_lang = tf.py_function(func=self.encode,
                                          inp=[pt, en],
                                          Tout=[tf.int64, tf.int64])
        pt_lang.set_shape([None])
        en_lang.set_shape([None])
        return pt_lang, en_lang

    def create_masks(self, batch):
        """creates all masks for training/validation"""
        def create_padding_mask(seq):
            """creates padding mask"""
            mask = tf.cast(tf.math.equal(seq, 0), tf.float32)
            return mask[:, tf.newaxis, tf.newaxis, :]

        def create_look_ahead_mask(size):
            """creates look ahead mask"""
            mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
            return mask

        def create_masks_decoder(tar):
            """creates all masks for training/validation"""
            look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
            dec_target_padding_mask = create_padding_mask(tar)
            combined_mask = tf.maximum(dec_target_padding_mask,
                                       look_ahead_mask)
            return combined_mask

        enc_padding_mask = create_padding_mask(batch[0])
        dec_padding_mask = create_padding_mask(batch[0])
        look_ahead_mask = create_look_ahead_mask(tf.shape(batch[1])[1])
        dec_target_padding_mask = create_padding_mask(batch[1])
        combined_mask = tf.maximum(dec_target_padding_mask,
                                   look_ahead_mask)
        return enc_padding_mask, combined_mask, dec_padding_mask

In [31]:
tf.compat.v1.set_random_seed(0)
data = Dataset(32, 40)
for inputs, target in data.data_train.take(1):
    print(data.create_masks(inputs, target))

2023-08-17 15:16:11.733229: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 12246 of 32768
2023-08-17 15:16:21.733139: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 28146 of 32768
2023-08-17 15:16:24.368854: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:417] Shuffle buffer filled.
2023-08-17 15:16:24.390258: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


TypeError: create_masks() takes 2 positional arguments but 3 were given